# Make Gif for this

In [7]:
import numpy as np
import plotly.graph_objects as go
import imageio
import os

# -----------------------------------
# Step 1: Define the Polygonal Region
# -----------------------------------
# Vertices of the feasible polygon (a quadrilateral)
polygon_vertices = np.array([
    [0, 0],
    [3, 0],
    [2, 3],
    [0, 4],
    [0, 0]  # Closing the polygon
])

# Linear objective function: x1 + x2 = z
c1, c2 = 1, 1  # Objective coefficients

# Define z-values for animation
z_values = range(1, 7)  # Integer contour levels from 1 to 8

# -----------------------------------
# Step 2: Generate Contour Lines and Frames
# -----------------------------------
# Create a folder for frames
if not os.path.exists("frames"):
    os.makedirs("frames")

# Function to compute the contour line in the non-negative orthant
def contour_line_in_orthant(z):
    # The line x1 + x2 = z intersects the axes at:
    # - (x1 = z, x2 = 0) on the x1-axis
    # - (x1 = 0, x2 = z) on the x2-axis
    return np.array([
        [z, 0],  # Intersection with x1-axis
        [0, z]   # Intersection with x2-axis
    ])

# Keep track of dashed lines
previous_contours = []

# Generate frames
for i, z in enumerate(z_values):
    fig = go.Figure()

    # Plot the polygon (feasible region)
    fig.add_trace(go.Scatter(
        x=polygon_vertices[:, 0],
        y=polygon_vertices[:, 1],
        fill="toself",
        line=dict(color="blue", width=2),
        name="Feasible Region"
    ))

    # Plot previous dashed contour lines
    for prev_z, prev_line in previous_contours:
        fig.add_trace(go.Scatter(
            x=prev_line[:, 0],
            y=prev_line[:, 1],
            mode="lines",
            line=dict(dash="dash", color="orange", width=2),
            name=f"z = {prev_z} (dashed)"
        ))

    # Compute the current contour line in the orthant
    contour_line = contour_line_in_orthant(z)

    # Add the current contour line
    fig.add_trace(go.Scatter(
        x=contour_line[:, 0],
        y=contour_line[:, 1],
        mode="lines",
        line=dict(color="red", width=3),
        name=f"z = {z}"
    ))

    # Save this contour line for future frames as dashed
    previous_contours.append((z, contour_line))

    # Update layout
    fig.update_layout(
        title=f"Linear Objective Optimization: x1 + x2 = z (z = {z})",
        xaxis=dict(range=[-0.5, 5], title="x1"),
        yaxis=dict(range=[-0.5, 5], title="x2"),
        showlegend=True
    )

    # Save the current frame as an image
    fig.write_image(f"frames/frame_{i}.png")

# -----------------------------------
# Step 3: Combine Frames into a GIF
# -----------------------------------
images = []
for i in range(len(z_values)):
    images.append(imageio.imread(f"frames/frame_{i}.png"))

output_gif = "linear_optimization.gif"
imageio.mimsave(output_gif, images, duration=1.2)

print(f"Animation saved as {output_gif}")


Animation saved as linear_optimization.gif


# Make interactive version of this

In [6]:
import numpy as np
import plotly.graph_objects as go

# -----------------------------------
# Step 1: Define the Polygonal Region
# -----------------------------------
# Vertices of the feasible polygon (a quadrilateral)
polygon_vertices = np.array([
    [0, 0],
    [3, 0],
    [2, 3],
    [0, 4],
    [0, 0]  # Closing the polygon
])

# Linear objective function: x1 + x2 = z
z_values = range(1, 7)  # Integer contour levels from 1 to 6

# -----------------------------------
# Step 2: Compute Contour Lines
# -----------------------------------
# Function to compute the contour line in the non-negative orthant
def contour_line_in_orthant(z):
    return [
        [z, 0],  # Intersection with x1-axis
        [0, z]   # Intersection with x2-axis
    ]

# -----------------------------------
# Step 3: Initialize the Base Figure
# -----------------------------------
# Base figure setup
fig = go.Figure()

# Static feasible region
fig.add_trace(go.Scatter(
    x=polygon_vertices[:, 0],
    y=polygon_vertices[:, 1],
    fill="toself",
    line=dict(color="blue", width=2),
    name="Feasible Region"
))

# -----------------------------------
# Step 4: Build Frames and Slider
# -----------------------------------
frames = []
steps = []

# Track all previous dashed lines
previous_lines = []

for z in z_values:
    frame_traces = []

    # Feasible region (re-added for each frame)
    frame_traces.append(go.Scatter(
        x=polygon_vertices[:, 0],
        y=polygon_vertices[:, 1],
        fill="toself",
        line=dict(color="blue", width=2),
        name="Feasible Region"
    ))

    # Add previous dashed lines
    for prev_z, prev_line in previous_lines:
        frame_traces.append(go.Scatter(
            x=[prev_line[0][0], prev_line[1][0]],
            y=[prev_line[0][1], prev_line[1][1]],
            mode="lines",
            line=dict(dash="dash", color="orange", width=2),
            name=f"z = {prev_z} (dashed)"
        ))

    # Compute and add the current contour line
    current_line = contour_line_in_orthant(z)
    frame_traces.append(go.Scatter(
        x=[current_line[0][0], current_line[1][0]],
        y=[current_line[0][1], current_line[1][1]],
        mode="lines",
        line=dict(color="red", width=3),
        name=f"z = {z} (current)"
    ))

    # Add current line to dashed list for future frames
    previous_lines.append((z, current_line))

    # Create a frame with all the traces
    frames.append(go.Frame(
        data=frame_traces,
        name=f"z = {z}"
    ))

    # Add a step to the slider
    steps.append(dict(
        method="animate",
        args=[[f"z = {z}"], {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}],
        label=str(z)
    ))

# -----------------------------------
# Step 5: Add Frames and Slider to the Figure
# -----------------------------------
fig.update(frames=frames)

# Slider definition
sliders = [dict(
    active=0,
    currentvalue={"prefix": "z = ", "font": {"size": 16}},
    steps=steps
)]

# Final layout updates
fig.update_layout(
    sliders=sliders,
    xaxis=dict(range=[-0.5, 5], title="x1"),
    yaxis=dict(range=[-0.5, 5], title="x2"),
    title="Linear Objective Optimization: x1 + x2 = z",
    showlegend=True
)

# Show the figure
fig.show()
